# Day 39: Model Performance Monitoring

In this lab, we will monitor a machine learning model observing a stream of data.
We will simulate **Concept Drift**, where the data distribution changes over time, causing the model's accuracy to degrade.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.observability.monitor.py import ModelMonitor

In [ ]:
# Fix import if failed (monitor.py module name might be just 'monitor' depending on how python resolved it in previous steps of this session)
try:
    from src.observability.monitor import ModelMonitor
except ImportError:
    # Fallback/Retry logic handled by user context usually
    pass

## 1. Simulate Data Stream

We create a stream of 1000 batches. 
At batch 50, the concept changes (the decision boundary flips).

In [ ]:
np.random.seed(42)

def generate_batch(batch_size, concept_id=0):
    X = np.random.randn(batch_size, 2)
    if concept_id == 0:
        # Concept 0: x[0] + x[1] > 0
        y = ((X[:, 0] + X[:, 1]) > 0).astype(int)
    else:
        # Concept 1 (Drift): x[0] + x[1] < 0 (Flip)
        y = ((X[:, 0] + X[:, 1]) < 0).astype(int)
    return X, y

# Pre-train model on Concept 0
X_train, y_train = generate_batch(200, concept_id=0)
model = LogisticRegression()
model.fit(X_train, y_train)

print("Initial Model Accuracy:", model.score(X_train, y_train))

## 2. Run Monitoring Loop

We process batches. After batch 50, we switch to Concept 1.

In [ ]:
monitor = ModelMonitor(window_size=10)
drift_alerted = False

accuracies = []

for i in range(100):
    # Introduce drift at Batch 50
    concept = 0 if i < 50 else 1
    
    X_batch, y_true = generate_batch(50, concept_id=concept)
    y_pred = model.predict(X_batch)
    
    monitor.update(y_true, y_pred, batch_id=i)
    
    # Check drift
    status = monitor.check_drift(threshold=0.6)
    
    accuracies.append(status['current_metric'])
    
    if status['drift_detected'] and not drift_alerted:
        print(f"[ALERT] Drift detected at Batch {i}! Accuracy: {status['current_metric']:.2f}")
        drift_alerted = True

## 3. Visualize Performance Decay

We plot the moving average accuracy.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(accuracies, label='Moving Average Accuracy')
plt.axvline(x=50, color='r', linestyle='--', label='Drift Start')
plt.axhline(y=0.6, color='g', linestyle=':', label='Threshold')
plt.title("Model Performance Over Time (Concept Drift)")
plt.xlabel("Batch ID")
plt.ylabel("Accuracy")
plt.legend()
plt.show()